### Text Selection Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
#path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Text Selection/Result/2-Text Selection Data Merge"

In [9]:
df_word = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx")
df_word = df_word.iloc[1000:1005,]
df_word

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
1000,arka,69809,"['arka tarafta', 'arka tarafa', 'arka kapı', '...","['arka tarafta bir', 'bir arka kapı', 'arka ta...","['arka kapısı var mı', 'buranın arka kapısı va...","['buranın arka kapısı var mı', 'onlar sadece a...",['arka kapısında tv seti çalarken yakalamışlar...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...,['herifi dışarı attığım gibi arka koltuğa geçt...
1001,hoşuma,69776,"['hoşuma gitti', 'hepinizi kutsasın', 'bu hoşu...","['hiç hoşuma gitmedi', 'çok hoşuma gitti', 'bu...","['bu hiç hoşuma gitmedi', 'her şeyi bilen biri...","['bu iş hiç hoşuma gitmedi', 'bu benim de hoşu...","['bana şaplak atmana izin vermek hoşuma', 'şap...",['bana şaplak atmana izin vermek hoşuma gider'...,['tvde diğer ülkeleri falan görünce çok hoşuma...,['gece gelip çattı ve durum daha baştan hoşuma...,['büyük gece gelip çattı ve durum daha baştan ...
1002,üçüncü,69620,"['üçüncü bir', 'üçüncü kat', 've üçüncü', 'bu ...","['üçüncü ve dördüncü', 'bu hafta üçüncü', 'üçü...","['güney üçüncü sıra temiz', 'bu hafta üçüncü k...","['kuzey üçüncü sıra temiz sayım', 'üçüncü bir ...",['üçüncü mevkii yolcuları sağlık kontrolü için...,['üçüncü mevkii yolcuları sağlık kontrolü için...,['bütün üçüncü mevkii yolcuları sağlık kontrol...,['bütün üçüncü mevkii yolcuları sağlık kontrol...,['bütün üçüncü mevkii yolcuları sağlık kontrol...
1003,sırada,69586,"['sırada ne', 'o sırada', 'arada sırada', 'bu ...","['sırada ne var', 'sırada sen varsın', 'tam o ...","['peki sırada ne var', 'şimdi sırada ne var', ...","['top şu sırada sağ tarafta', 'arada sırada gü...","['bizi sırada ne tür bir macera', 'sırada ne t...","['sırada ne tür bir macera bekliyor acaba', 'b...",['bizi sırada ne tür bir macera bekliyor acaba...,['bu arkadaşın arkadaşa yapmayacağı şeyler lis...,['çünkü bu arkadaşın arkadaşa yapmayacağı şeyl...
1004,gidecek,69388,"['gidecek misin', 'yolunda gidecek', 'gidecek ...","['şey yolunda gidecek', 'çok hoşuna gidecek', ...","['geri getir olur mu', 'gidecek bir yerim yok'...","['gidecek başka bir yerim yok', 'gidecek bir y...","['burada bana eşlik etmen hoşuma gidecek', 'me...","['bu gece oraya bir adam uğradı mı', 'helikopt...",['adam gelecek ve sözde kayıp bavulunu alıp gi...,['gidecek birini rehin aldı ve ona kibar mı da...,['mahkemeye gidecek birini rehin aldı ve ona k...


In [10]:
df_twogram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx")
df_twogram = df_twogram.head(200)
df_twogram

,twogram,frequency,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir şey,859944,"['bir şey yok', 'bir şey var', 'başka bir şey'...","['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'sana bir şey sorabil...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,"['bu iyi bir şey mi kötü bir şey mi', 'daha ön...",['bana sadece içinde şeker olmayan bir şey ver...
1,değil mi,585879,"['öyle değil mi', 'biliyorsun değil mi', 'güze...","['de öyle değil mi', 'bir şey değil mi', 'çok ...","['sence de öyle değil mi', 'sizce de öyle deği...","['bu iyi bir şey değil mi', 'ne demek istediği...","['o kadar da kötü değildi değil mi', 'o kadar ...",['bir kadının şarkı söylemesi oldukça şaşırtıc...,['evimde bir kadının şarkı söylemesi oldukça ş...,['ama bugün yeni bir maç var galiba değil mi m...
2,ben de,377765,"['ben de seni', 'ben de öyle', 've ben de', 'e...","['ben de seni seviyorum', 'ben de öyle düşünmü...","['ben de seni gördüğüme sevindim', 'evet ben d...","['ben de en az senin kadar', 'ben de sana aynı...","['ben de sizi karı koca ilan ediyorum', 'ben d...",['siobhan intihar etti ben de onun yerine geçt...,['siz formları alırsınız ben de onları hazırla...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,teşekkür ederim,370619,"['çok teşekkür ederim', 'için teşekkür ederim'...","['için çok teşekkür ederim', 'geldiğiniz için ...","['her şey için teşekkür ederim', 'zaman ayırdı...","['her şey için çok teşekkür ederim', 'beni kab...",['olmama izin verdiğin için tekrar teşekkür ed...,['gibi olmama izin verdiğin için tekrar teşekk...,['kendim gibi olmama izin verdiğin için tekrar...,['önce parfüm mağazasından alınmış video kaydı...
4,ne oldu,322758,"['sonra ne oldu', 'ne oldu sana', 'sana ne old...","['bil bakalım ne oldu', 'sana ne oldu böyle', ...","['az önce ne oldu öyle', 'sonra ne oldu biliyo...","['ne oldu bir sorun mu var', 'az önce ne oldu ...",['gelecek hakkındaki bilgileri reddetme konuşm...,['insanlık nereye gidiyoruz ve niçin soruların...,['ya insanlık nereye gidiyoruz ve niçin sorula...,['burnuna ne oldu gittes yoksa kapı yerine bac...
...,...,...,...,...,...,...,...,...,...,...
195,her neyse,37501,"['her neyse bu', 'her neyse işte', 'evet her n...","['ya da her neyse', 'bu şey her neyse', 'da ad...","['ya da adı her neyse', 'ya da her neyse işte'...","['her neyse bu konuyu daha fazla', 'her neyse ...","['her neyse bir doktora benziyor değil mi', 'g...",['öldüren her neyse üzerinden çok zaman geçmiş...,['öldüren her neyse üzerinden çok zaman geçmiş...,['onları öldüren her neyse üzerinden çok zaman...
196,ne güzel,37424,"['ne güzel bir', 'görmek ne güzel', 'aman ne g...","['seni görmek ne güzel', 'ne güzel değil mi', ...","['seni tekrar görmek ne güzel', 'sizi tekrar g...","['ne güzel bir gün değil mi', 'tekrar işler va...",['sizi tekrar işler vaziyette görmek ne güzel'...,['luke sizi tekrar işler vaziyette görmek ne g...,['efendi luke sizi tekrar işler vaziyette görm...,['joshua baban seni ne güzel bir yere getirmiş...
197,çok teşekkür,37407,"['çok teşekkür ederim', 'için çok teşekkür', '...","['için çok teşekkür ederim', 'çok teşekkür ede...","['geldiğiniz için çok teşekkür ederim', 'geldi...","['her şey için çok teşekkür ederim', 'bunu yap...",['beni davet ettiğiniz için çok teşekkür ederi...,['bana gönderdiğiniz her şey için çok teşekkür...,['biliyor musunuz ben ben size çok teşekkür et...,['çok teşekkür ederim bir şişe sıcak su isters...
198,öyle bir,36881,"['öyle bir şey', 'yok öyle bir', 'ben öyle bir...","['öyle bir şey yok', 'öyle bir şey demedim', '...","['ben öyle bir şey demedim', 'ya da öyle bir ş...","['neden öyle bir şey yapayım ki', 'ya da öyle ...","['öyle bir niyetin varsa hiç durma duvarda', '...",['öyle bir şey yaptın mı bütün dostların sende...,['öyle bir şey yaptın mı bütün dostların sende...,['şimdi sen de öyle bir adam olup olmadığına k...


In [11]:
df_threegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx")
df_threegram = df_threegram.head(200)
df_threegram

,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir şey yok,113165,"['diye bir şey yok', 'başka bir şey yok', 'bir...","['biz diye bir şey yok', 'başka bir şey yok mu...","['bir şey yok bir şey yok', 'yok bir şey yok b...","['söylemek istediğin başka bir şey yok mu', 'b...",['birbirimiz olmadan ikimizin de yapacağı bir ...,['ölmeden önce de aşıktım ve yapabileceğim bir...,['o ölmeden önce de aşıktım ve yapabileceğim b...
1,bir şey var,110455,"['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'yapabileceğim bir şe...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['ev sahibi babamız için bende özel bir şey va...,['kaosun içinde düzen için mücadele eden tek b...
2,bu da ne,89463,"['bu da ne demek', 'bu da ne böyle', 'bu da ne...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu da ne demek oluyor şimdi', 'bu da ne deme...","['bu da ne biçim bir soru böyle', 'ama bu da n...","['alınmayın ama bu da ne demek oluyor şimdi', ...",NaN,['bu da ne olursa olsun oynamaya karar verdiği...
3,her şey yolunda,75968,"['her şey yolunda mı', 'her şey yolunda gidece...","['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda her şey yolunda', 'merak etm...",['helikopterler bizi koruyacak her şey yolunda...,"['her şey yolunda giderse çok yakında bir ev',...",['her şey yolunda giderse çok yakında bir ev a...,['dostlar acil bir durum yaşadık ama şimdi her...
4,başka bir şey,75193,"['başka bir şey var', 'başka bir şey yok', 'ba...","['başka bir şey var mı', 'başka bir şey daha v...","['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['kılıcını ve onun kılıcını düşün başka bir şe...,['sabırsızlıkla beklediğim zevkle yapacağım ba...
...,...,...,...,...,...,...,...,...,...
195,bu değil mi,10578,"['da bu değil mi', 'şey bu değil mi', 'de bu d...","['olan da bu değil mi', 'önemli olan bu değil ...","['önemli olan da bu değil mi', 'senin istediği...","['gördüğün en güzel şey bu değil mi', 'seni ko...","['kazanması çok zor bir kumar bu değil mi', 'y...",['ama kazanması çok zor bir kumar bu değil mi'...,['evet ama kazanması çok zor bir kumar bu deği...
196,kadar da kötü,10556,"['o kadar da kötü', 'kadar da kötü değil', 'ka...","['o kadar da kötü değil', 'o kadar da kötü değ...","['bu o kadar da kötü değil', 'kadar da kötü de...","['o kadar da kötü değildi değil mi', 'o kadar ...","['bu o kadar da kötü bir şey değil', 'bu o kad...",['bir düşün o kadar da kötü değilsin ha zenci'...,['ve o bana hayatın o kadar da kötü olmadığını...
197,ne olduğunu biliyor,10553,"['ne olduğunu biliyor musun', 'bunun ne olduğu...","['bunun ne olduğunu biliyor musun', 'bunun ne ...","['bunun ne olduğunu biliyor musunuz bay', 'may...",['bunun ne olduğunu biliyor musunuz bay fische...,['şikayet edilecek tek şeyin ne olduğunu biliy...,['bir altın madeni yapan şeyin ne olduğunu bil...,['ona göre şikayet edilecek tek şeyin ne olduğ...
198,beni rahat bırak,10545,"['lütfen beni rahat bırak', 've beni rahat bır...","['beni rahat bırak tamam mı', 'beni rahat bıra...","['kapa çeneni ve beni rahat bırak', 'beni raha...",['lütfen beni rahat bırak yoksa polis çağırırı...,['şimdi beni rahat bırak tuvalet temizliyorum ...,['oraya gitmek istiyorsan git ama beni rahat b...,['şimdi beni rahat bırak tuvalet temizliyorum ...


In [12]:
df_fourgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx")
df_fourgram = df_fourgram.head(200)
df_fourgram

,fourgram,frequency,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir şey var mı,41773,"['başka bir şey var mı', 'yapabileceğim bir şe...","['söylemek istediğin bir şey var mı', 'için ya...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['sizin için yapabileceğim başka bir şey var m...,['tanrı aşkına çalışmak için yaşamak diye bir ...
1,her şey yolunda mı,31126,"['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda mı diye bakmak', 'her şey yo...","['her şey yolunda mı diye bakmak istedim', 'he...","['git de her şey yolunda mı bir bak', 'hadi gi...",['hadi git de her şey yolunda mı bir bak'],['yukarı gelip her şey yolunda mı diye kontrol...
2,burada ne işin var,21993,"['senin burada ne işin var', 'burada ne işin v...","['o zaman burada ne işin var', 'bu saatte bura...","['gecenin bu saatinde burada ne işin var', 'on...",['o kadar paran varsa burada ne işin var'],NaN,NaN
3,bir sorun mu var,21423,"['bir sorun mu var efendim', 'ilgili bir sorun...","['bir sorun mu var memur bey', 'ile ilgili bir...","['yerçekimi ile ilgili bir sorun mu var', 'dem...",['küresel yerçekimi ile ilgili bir sorun mu va...,['neden demin geldiğimde bir sorun mu var diye...,['merak ediyorum da acaba bende bir sorun mu v...
4,ben de seni seviyorum,17338,"['ben de seni seviyorum baba', 'ben de seni se...","['ben de seni seviyorum biliyorsun değil', 'be...","['ben de seni seviyorum biliyorsun değil mi', ...","['bu çok güzel çünkü ben de seni seviyorum', '...",NaN,['ben de seni seviyorum tatlım hem de her şeyd...
...,...,...,...,...,...,...,...,...
195,daha iyi değil mi,2362,"['böyle daha iyi değil mi', 'böylesi daha iyi ...","['ama parası daha iyi değil mi', 'çay yapmakta...","['böylesi herkes için daha iyi değil mi', 'bir...",['sence buzdolabı onlar için daha iyi değil mi...,['sakin durup işi dışarıda bitirmek daha iyi d...,['şimdi sakin durup işi dışarıda bitirmek daha...
196,tek bir şey var,2360,"['yapılacak tek bir şey var', 'yapacak tek bir...","['tek bir şey var o da', 'bilmen gereken tek b...","['söylemem gereken yalnızca tek bir şey var', ...","['düzen için mücadele eden tek bir şey var', '...",['içinde düzen için mücadele eden tek bir şey ...,['kaosun içinde düzen için mücadele eden tek b...
197,çok kötü bir şey,2350,"['çok kötü bir şey oldu', 'çok kötü bir şey ya...","['ben çok kötü bir şey yaptım', 'çok kötü bir ...","['gece başına çok kötü bir şey geldi', 'dün ge...","['zavallı dün gece başına çok kötü bir şey', '...",['zavallı dün gece başına çok kötü bir şey gel...,['ama şimdi çok kötü bir şey olduğunu söylemek...
198,çok önemli bir şey,2348,"['çok önemli bir şey var', 'çok önemli bir şey...","['gereken çok önemli bir şey var', 'söylemem g...","['söylemem gereken çok önemli bir şey var', 's...",['sana söylemem gereken çok önemli bir şey var...,['düşün ya hayatında çok önemli bir şey yapaca...,['sadece düşün ya hayatında çok önemli bir şey...


In [13]:
df_fivegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx")
df_fivegram = df_fivegram.head(200)
df_fivegram

,fivegram,frequency,sixgram,sevengram,eightgram,ninegram,tengram
0,başka bir şey var mı,14104,"['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['sizin için yapabileceğim başka bir şey var m...,['affedersiniz bayan sizin için yapabileceğim ...
1,bu da ne demek oluyor,10205,"['bu da ne demek oluyor şimdi', 'bu da ne deme...","['ama bu da ne demek oluyor şimdi', 'alınmayın...",['alınmayın ama bu da ne demek oluyor şimdi'],NaN,['bu da ne demek oluyor yargılamak gibi bir şe...
2,o kadar da kötü değil,7012,"['bu o kadar da kötü değil', 'o kadar da kötü ...","['o kadar da kötü değil değil mi', 'hadi ama o...","['bu o kadar da kötü değil değil mi', 'ama o k...",['hadi ama pascal burası o kadar da kötü değil...,['hem prenses olmak o kadar da kötü değil deği...
3,sence de öyle değil mi,6305,"['görünüyor sence de öyle değil mi', 'var senc...","['iyi gidiyor sence de öyle değil mi', 'zamand...","['hoş bir vadi sence de öyle değil mi', 'çok d...","['burası hoş bir vadi sence de öyle değil mi',...",['gelecek geçmişten çok daha ilginç sence de ö...
4,sana bir şey sorabilir miyim,6224,"['hey sana bir şey sorabilir miyim', 'baba san...",['kimseye söylemezsen sana bir şey sorabilir m...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
195,bu çok iyi bir fikir,841,"['bence bu çok iyi bir fikir', 'evet bu çok iy...","['bu çok iyi bir fikir olmayabilir doktor', 'b...",NaN,NaN,['bu çok iyi bir fikir olurdu hayata geçirilmi...
196,ben o kadar emin değilim,826,['ama ben o kadar emin değilim'],NaN,NaN,NaN,NaN
197,herhangi bir fikrin var mı,820,"['hakkında herhangi bir fikrin var mı', 'dair ...","['olduğuna dair herhangi bir fikrin var mı', '...",['ne olduğuna dair herhangi bir fikrin var mı'...,['olmasının dışında kudüs hakkında herhangi bi...,['senin olmasının dışında kudüs hakkında herha...
198,sen yanlış bir şey yapmadın,819,"['sen yanlış bir şey yapmadın ki', 'tabii sen ...",['sen yanlış bir şey yapmadın tamam mı'],"['sen yanlış bir şey yapmadın ama bu beni', 'e...",['sen yanlış bir şey yapmadın ama bu beni ilgi...,NaN


In [14]:
df_sixgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx")
df_sixgram = df_sixgram.head(200)
df_sixgram

,sixgram,frequency,sevengram,eightgram,ninegram,tengram
0,sana söylemem gereken bir şey var,3646,"['ama sana söylemem gereken bir şey var', 'amc...","['benim de sana söylemem gereken bir şey var',...",['sana söylemem gereken bir şey var ama nasıl ...,['sana söylemem gereken bir şey var ama nasıl ...
1,bir iki üç dört beş altı,2025,"['bir iki üç dört beş altı yedi', 've bir iki ...","['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,söylemek istediğin bir şey var mı,1794,"['bana söylemek istediğin bir şey var mı', 'bi...",['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,daha iyi bir fikrin var mı,1792,"['senin daha iyi bir fikrin var mı', 'peki dah...","['peki daha iyi bir fikrin var mı şerif', 'lan...",NaN,['benim dahil olmadığım daha iyi bir fikrin va...
4,bunun iyi bir fikir olduğunu sanmıyorum,1743,['ama bunun iyi bir fikir olduğunu sanmıyorum'...,['hayır hayır bunun iyi bir fikir olduğunu san...,NaN,NaN
...,...,...,...,...,...,...
195,söylemem gereken önemli bir şey var,173,"['sana söylemem gereken önemli bir şey var', '...",['ayrıca sana söylemem gereken önemli bir şey ...,NaN,NaN
196,bu işte bir bit yeniği var,173,"['bu işte bir bit yeniği var dostum', 'bence b...",NaN,NaN,['sen de biliyorsun ki bu işte bir bit yeniği ...
197,bunun senin için ne kadar zor,173,"['bunun senin için ne kadar zor olduğunu', 'la...",['bunun senin için ne kadar zor olduğunu biliy...,['lara bunun senin için ne kadar zor olduğunu ...,['bunun senin için ne kadar zor olduğunu hayal...
198,hey hey hey hey hey hey,173,['hey hey hey hey hey hey hey'],['hey hey hey hey hey hey hey hey'],['hey hey hey hey hey hey hey hey hey'],['hey hey hey hey hey hey hey hey hey hey']


In [15]:
df_sevengram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx")
df_sevengram = df_sevengram.head(200)
df_sevengram

,sevengram,frequency,eightgram,ninegram,tengram
0,bir iki üç dört beş altı yedi,1455,"['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,iki üç dört beş altı yedi sekiz,1080,"['bir iki üç dört beş altı yedi sekiz', 'iki ü...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,bana söylemek istediğin bir şey var mı,752,['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,yedi altı beş dört üç iki bir,596,"['sekiz yedi altı beş dört üç iki bir', 'yedi ...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
4,sekiz yedi altı beş dört üç iki,551,"['dokuz sekiz yedi altı beş dört üç iki', 'sek...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
...,...,...,...,...,...
195,hiç bu kadar güzel bir şey görmemiştim,69,['önce hiç bu kadar güzel bir şey görmemiştim'...,['daha önce hiç bu kadar güzel bir şey görmemi...,NaN
196,söylemek istediğim o kadar çok şey var,69,"['söylemek istediğim o kadar çok şey var ki', ...",['sana söylemek istediğim o kadar çok şey var ...,NaN
197,adı uğruna bana doğru yollarda öncülük eder,68,['tazeler adı uğruna bana doğru yollarda öncül...,NaN,NaN
198,ve kutsal ruh adına seni vaftiz ediyorum,68,['oğul ve kutsal ruh adına seni vaftiz ediyorum'],['baba oğul ve kutsal ruh adına seni vaftiz ed...,NaN


In [16]:
df_eightgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx")
df_eightgram = df_eightgram.head(200)
df_eightgram

,eightgram,frequency,ninegram,tengram
0,bir iki üç dört beş altı yedi sekiz,1060,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,dokuz sekiz yedi altı beş dört üç iki,513,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
2,sekiz yedi altı beş dört üç iki bir,497,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
3,iki üç dört beş altı yedi sekiz dokuz,411,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
4,üç dört beş altı yedi sekiz dokuz on,329,"['iki üç dört beş altı yedi sekiz dokuz on', '...",['bir iki üç dört beş altı yedi sekiz dokuz on...
...,...,...,...,...
195,beni bekleyen pek çok şey daha olduğunu biliyorum,40,['hayatta beni bekleyen pek çok şey daha olduğ...,NaN
196,eğer buradan bir gün çıkacak olursan bana bir,40,['eğer buradan bir gün çıkacak olursan bana bi...,['eğer buradan bir gün çıkacak olursan bana bi...
197,istediğim buraya yeni kitaplar getirmeyi bay d...,40,['demek istediğim buraya yeni kitaplar getirme...,NaN
198,bir buz devri bin yıllık bir dağ oluşturmuş,40,['bir buz devri bin yıllık bir dağ oluşturmuş ...,NaN


In [17]:
df_ninegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx")
df_ninegram = df_ninegram.head(200)
df_ninegram

,ninegram,frequency,tengram
0,dokuz sekiz yedi altı beş dört üç iki bir,463,['on dokuz sekiz yedi altı beş dört üç iki bir...
1,bir iki üç dört beş altı yedi sekiz dokuz,405,['bir iki üç dört beş altı yedi sekiz dokuz on...
2,iki üç dört beş altı yedi sekiz dokuz on,287,['bir iki üç dört beş altı yedi sekiz dokuz on']
3,on dokuz sekiz yedi altı beş dört üç iki,277,['on dokuz sekiz yedi altı beş dört üç iki bir']
4,i t a r i s y e n,128,['g i t a r i s y e n']
...,...,...,...
195,hala fabrikada çalışıyor ben de bir güzellik s...,31,['annem hala fabrikada çalışıyor ben de bir gü...
196,her şey yoluna girdiğine göre konyak içmeye de...,31,['her şey yoluna girdiğine göre konyak içmeye ...
197,işi para için yapıyorum ve iyi bir ücret bekli...,31,['bu işi para için yapıyorum ve iyi bir ücret ...
198,sözünün eri ve nedenleri dinleyen mütevazi bir...,31,['onu sözünün eri ve nedenleri dinleyen müteva...


#### Sending Data To Firestore

##### Photo Data

In [18]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Text Data

##### Word

In [18]:
word = df_word["word"].to_list()
frequency_list = df_word["frequency"].to_list()
twogram_list = df_word["twogram"].to_list()
threegram_list = df_word["threegram"].to_list()
fourgram_list = df_word["fourgram"].to_list()
fivegram_list = df_word["fivegram"].to_list()
sixgram_list = df_word["sixgram"].to_list()
sevengram_list = df_word["sevengram"].to_list()
eightgram_list = df_word["eightgram"].to_list()
ninegram_list = df_word["ninegram"].to_list()
tengram_list = df_word["tengram"].to_list()

In [19]:
firestore_word_data = zip(word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [20]:
#word_data_list = []
#for word,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
#    data = {f"{word}":{"ninegram":ninegram_list,"tengram":tengram_list}}
#    
#    word_data_list.append({f"{word}":data})

In [21]:
word_dict = {}
for word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
    dict_var = {f"{word}":{"photo":"","frequency":frequency_list,"twogram":twogram_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[""]}}
    word_dict.update(dict_var)     

In [22]:
#word_dict

In [28]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").set(word_dict)

update_time {
  seconds: 1670315613
  nanos: 765935000
}

In [32]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").add(word_dict)

##### Update

In [36]:
update_word = "gidecek"

In [37]:
dict_update = {f"{update_word}.photo":"www.store1.com",f"{update_word}.other":["gitti",{"zıt":"gelmek"}]}

In [38]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").update(dict_update)

update_time {
  seconds: 1670316106
  nanos: 15609000
}

##### Twogram

In [23]:
twogram = df_word["twogram"].to_list()
frequency_list = df_word["frequency"].to_list()
threegram_list = df_word["threegram"].to_list()
fourgram_list = df_word["fourgram"].to_list()
fivegram_list = df_word["fivegram"].to_list()
sixgram_list = df_word["sixgram"].to_list()
sevengram_list = df_word["sevengram"].to_list()
eightgram_list = df_word["eightgram"].to_list()
ninegram_list = df_word["ninegram"].to_list()
tengram_list = df_word["tengram"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1 Level", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2 Level", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1 Level", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2 Level", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1 Level", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2 Level", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1669369148
  nanos: 63795000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass